In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

Loading imports


In [3]:
torch.cuda.set_device(2)

In [4]:
dls_feat = get_3d_dls_feat(Meta.df_comb, path=path_feat_384avg_adj, bs=32)

## Model

In [5]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer
class LinHead(nn.Module):
    def __init__(self):
        super().__init__()
        d_model = 1024
        self.flat = nn.Sequential(AdaptiveConcatPool2d(), Flatten())
        self.hook = ReshapeBodyHook(self.flat)
        self.head = nn.Sequential(nn.Linear(d_model,6))
        
    def forward(self, x):
        x = self.flat(x)
        return self.head(x)

In [6]:
m = LinHead()
name = 'train3d_adj_feat_linear'
learn = get_learner(dls_feat, m, name=name)
learn.add_cb(DePadLoss())

In [7]:
# learn.summary()

## Training

In [ ]:
learn.lr_find()

In [8]:
do_fit(learn, 10, 1e-3)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.089357,0.091036,0.972184,0.948450,01:14
1,0.056461,0.076232,0.976279,0.953517,01:17
2,0.055003,0.082583,0.975564,0.950436,01:17
3,0.056352,0.075044,0.976409,0.954561,01:15
4,0.056004,0.074828,0.976720,0.955058,01:15
5,0.053732,0.075398,0.976519,0.954319,01:15
6,0.052981,0.074733,0.976741,0.954778,01:16
7,0.053664,0.074333,0.976871,0.955312,01:15
8,0.051987,0.074424,0.976752,0.954994,01:16
9,0.051747,0.074541,0.976750,0.954816,01:14


## Testing

In [9]:
learn.dls = get_3d_dls_feat(Meta.df_tst, path=path_feat_tst_384avg_adj, bs=32, test=True)

In [10]:
sub_fn = f'subm/{name}'
learn.load(f'runs/{name}-1')

In [11]:
preds,targs = learn.get_preds()
preds.shape, preds.min(), preds.max()

(torch.Size([121232, 6]), tensor(7.7020e-10), tensor(1.0000))

In [12]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [13]:
api.competition_submit(f'{sub_fn}.csv', name, 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:01<00:00, 14.2MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [14]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14373211,
 'totalBytes': 27277209,
 'date': '2020-02-04T02:24:49.077Z',
 'description': 'train3d_adj_feat_linear',
 'errorDescription': None,
 'fileName': 'train3d_adj_feat_linear.csv',
 'publicScore': '0.49515',
 'privateScore': '0.06029',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14373211/14373211.raw'}